In [1]:
# ═══════════════════════════════════════════════════════════════════
# 🦟 MOSQUITO PREDICTOR - COMPLETE BACKEND (ALL-IN-ONE)
# ═══════════════════════════════════════════════════════════════════

print("="*70)
print("🦟 MOSQUITO PREDICTOR - COMPLETE SETUP")
print("="*70)

# ─────────────────────────────────────────────────────────────────────
# STEP 1: Install Dependencies
# ─────────────────────────────────────────────────────────────────────
print("\n📦 Installing packages...")
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "flask", "flask-cors", "pillow"])
subprocess.check_call(["apt-get", "install", "-y", "-qq", "nodejs", "npm"])
subprocess.check_call(["npm", "install", "-g", "localtunnel"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print("✅ Packages installed!")

# ─────────────────────────────────────────────────────────────────────
# STEP 2: Upload Model
# ─────────────────────────────────────────────────────────────────────
print("\n📤 Please upload: mosquito_risk_model.keras")
from google.colab import files
uploaded = files.upload()

if 'mosquito_risk_model.keras' not in uploaded:
    print("❌ Model file not found!")
    raise Exception("Please upload mosquito_risk_model.keras")

print(f"✅ Model uploaded ({len(uploaded['mosquito_risk_model.keras'])/1024/1024:.2f} MB)")

# ─────────────────────────────────────────────────────────────────────
# STEP 3: Setup Database
# ─────────────────────────────────────────────────────────────────────
print("\n🗄️ Creating database...")
import sqlite3
from datetime import datetime

conn = sqlite3.connect('mosquito_predictor.db', check_same_thread=False)
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT, full_name TEXT, email TEXT UNIQUE,
    password TEXT, role TEXT, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS reports (
    id INTEGER PRIMARY KEY AUTOINCREMENT, user_id INTEGER, title TEXT,
    location TEXT, description TEXT, image_path TEXT, predicted_class TEXT,
    risk_level TEXT, confidence REAL, status TEXT DEFAULT 'Pending',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')

cursor.execute("INSERT OR IGNORE INTO users VALUES (1, 'Admin', 'admin@mosquito.com', 'admin123', 'admin', CURRENT_TIMESTAMP)")
cursor.execute("INSERT OR IGNORE INTO users VALUES (2, 'User', 'user@mosquito.com', 'user123', 'user', CURRENT_TIMESTAMP)")
conn.commit()

print("✅ Database ready!")

# ─────────────────────────────────────────────────────────────────────
# STEP 4: Load ML Model
# ─────────────────────────────────────────────────────────────────────
print("\n🤖 Loading model...")
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import io

model = load_model('mosquito_risk_model.keras', compile=False)
class_names = ['Breeding Place', 'Mosquito', 'Mosquito Swarm']
risk_mapping = {
    'Breeding Place': 'High Risk',
    'Mosquito': 'Moderate Risk',
    'Mosquito Swarm': 'High Risk'
}

def predict_mosquito_risk(image_file):
    try:
        img = Image.open(image_file if isinstance(image_file, str) else io.BytesIO(image_file))
        img = img.resize((224, 224))
        img_array = np.expand_dims(np.array(img) / 255.0, axis=0)
        predictions = model.predict(img_array, verbose=0)
        class_idx = np.argmax(predictions[0])
        return {
            'class_name': class_names[class_idx],
            'risk_level': risk_mapping[class_names[class_idx]],
            'confidence': round(float(predictions[0][class_idx] * 100), 2)
        }
    except:
        return {'class_name': 'Unknown', 'risk_level': 'Unknown', 'confidence': 0.0}

print("✅ Model loaded!")

# ─────────────────────────────────────────────────────────────────────
# STEP 5: Create Flask App
# ─────────────────────────────────────────────────────────────────────
print("\n⚙️ Creating Flask API...")
from flask import Flask, request, jsonify
from flask_cors import CORS
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
CORS(app)
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

print("✅ Flask app created!")

# ─────────────────────────────────────────────────────────────────────
# STEP 6: Add ALL API Endpoints
# ─────────────────────────────────────────────────────────────────────
print("\n🔌 Adding API endpoints...")

@app.route('/api/signup', methods=['POST'])
def signup():
    try:
        data = request.json
        cursor.execute('SELECT id FROM users WHERE email = ?', (data['email'],))
        if cursor.fetchone(): return jsonify({'error': 'Email exists'}), 409
        cursor.execute('INSERT INTO users (full_name, email, password, role) VALUES (?, ?, ?, ?)',
                      (data.get('full_name'), data['email'], data['password'], data.get('role', 'user')))
        conn.commit()
        return jsonify({'message': 'User registered', 'user_id': cursor.lastrowid}), 201
    except Exception as e: return jsonify({'error': str(e)}), 500

@app.route('/api/login', methods=['POST'])
def login():
    try:
        data = request.json
        cursor.execute('SELECT id, full_name, email, role FROM users WHERE email = ? AND password = ? AND role = ?',
                      (data['email'], data['password'], data['role']))
        user = cursor.fetchone()
        if user: return jsonify({'message': 'Login successful', 'user': {'id': user[0], 'name': user[1], 'email': user[2], 'role': user[3]}}), 200
        return jsonify({'error': 'Invalid credentials'}), 401
    except Exception as e: return jsonify({'error': str(e)}), 500

@app.route('/api/reports', methods=['POST'])
def create_report():
    try:
        user_id, title, location = request.form.get('user_id', 2), request.form.get('title', ''), request.form.get('location', '')
        description, image_path, predicted_class, risk_level, confidence = request.form.get('description', ''), None, None, None, None

        if 'image' in request.files:
            file = request.files['image']
            if file and allowed_file(file.filename):
                filename = secure_filename(f"{datetime.now().timestamp()}_{file.filename}")
                filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
                file.save(filepath)
                image_path, prediction = filename, predict_mosquito_risk(filepath)
                predicted_class, risk_level, confidence = prediction['class_name'], prediction['risk_level'], prediction['confidence']

        cursor.execute('INSERT INTO reports (user_id, title, location, description, image_path, predicted_class, risk_level, confidence, status) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)',
                      (user_id, title, location, description, image_path, predicted_class, risk_level, confidence, 'Pending'))
        conn.commit()
        return jsonify({'message': 'Report submitted', 'report_id': cursor.lastrowid,
                       'prediction': {'class': predicted_class, 'risk_level': risk_level, 'confidence': confidence} if predicted_class else None}), 201
    except Exception as e: return jsonify({'error': str(e)}), 500

@app.route('/api/reports', methods=['GET'])
def get_reports():
    try:
        user_id = request.args.get('user_id')
        cursor.execute('SELECT * FROM reports WHERE user_id = ? ORDER BY created_at DESC' if user_id else 'SELECT * FROM reports ORDER BY created_at DESC', (user_id,) if user_id else ())
        return jsonify({'reports': [{'id': r[0], 'title': r[2], 'location': r[3], 'predicted_class': r[6], 'risk_level': r[7], 'confidence': r[8], 'status': r[9], 'created_at': r[10]} for r in cursor.fetchall()]}), 200
    except Exception as e: return jsonify({'error': str(e)}), 500

@app.route('/api/dashboard/stats', methods=['GET'])
def get_stats():
    try:
        user_id, q = request.args.get('user_id'), f"WHERE user_id = {request.args.get('user_id')}" if request.args.get('user_id') else ""
        cursor.execute(f'SELECT COUNT(*) FROM reports {q}'); total = cursor.fetchone()[0]
        cursor.execute(f'SELECT COUNT(*) FROM reports {q} {"AND" if user_id else "WHERE"} status = "Pending"'); pending = cursor.fetchone()[0]
        cursor.execute(f'SELECT COUNT(*) FROM reports {q} {"AND" if user_id else "WHERE"} status = "In Progress"'); in_progress = cursor.fetchone()[0]
        cursor.execute(f'SELECT COUNT(*) FROM reports {q} {"AND" if user_id else "WHERE"} status = "Resolved"'); resolved = cursor.fetchone()[0]
        return jsonify({'total_reports': total, 'pending': pending, 'in_progress': in_progress, 'resolved': resolved}), 200
    except Exception as e: return jsonify({'error': str(e)}), 500

@app.route('/api/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy', 'model_loaded': model is not None}), 200

print("✅ API endpoints added!")

# ─────────────────────────────────────────────────────────────────────
# STEP 7: Start Localtunnel
# ─────────────────────────────────────────────────────────────────────
print("\n🌐 Starting localtunnel...")
import threading
import re
import time

tunnel_url = None
def run_tunnel():
    global tunnel_url
    process = subprocess.Popen(['lt', '--port', '5000'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    for line in process.stdout:
        if 'your url is:' in line.lower():
            match = re.search(r'https://[^\s]+', line)
            if match: tunnel_url = match.group(0); break

threading.Thread(target=run_tunnel, daemon=True).start()
time.sleep(5)

# ─────────────────────────────────────────────────────────────────────
# STEP 8: Start Flask Server
# ─────────────────────────────────────────────────────────────────────
print("\n🚀 Starting Flask server...")

def run_app():
    app.run(port=5000, debug=False, use_reloader=False, threaded=True)

threading.Thread(target=run_app, daemon=True).start()
time.sleep(3)


!curl https://loca.lt/mytunnelpassword

# ─────────────────────────────────────────────────────────────────────
# FINAL SUMMARY
# ─────────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("✅ MOSQUITO PREDICTOR - READY!")
print("="*70)

if tunnel_url:
    print(f"\n🔗 PUBLIC URL: {tunnel_url}")
    print(f"\n📝 Update your HTML files:")
    print(f"   const API_URL = '{tunnel_url}';")
    print(f"\n🔑 Tunnel Password (if asked): Run !curl https://loca.lt/mytunnelpassword")
else:
    print("\n⚠️ To get URL, run: !lt --port 5000")

print(f"\n🎯 Test Credentials:")
print(f"   User:  user@mosquito.com / user123")
print(f"   Admin: admin@mosquito.com / admin123")

print(f"\n🧪 Test Your API:")
if tunnel_url:
    print(f"   {tunnel_url}/api/health")

print("\n⚠️  KEEP THIS CELL RUNNING!")
print("="*70)

# Keep running
while True:
    time.sleep(10)


🦟 MOSQUITO PREDICTOR - COMPLETE SETUP

📦 Installing packages...
✅ Packages installed!

📤 Please upload: mosquito_risk_model.keras


Saving mosquito_risk_model.keras to mosquito_risk_model.keras
✅ Model uploaded (18.14 MB)

🗄️ Creating database...
✅ Database ready!

🤖 Loading model...
✅ Model loaded!

⚙️ Creating Flask API...
✅ Flask app created!

🔌 Adding API endpoints...
✅ API endpoints added!

🌐 Starting localtunnel...

🚀 Starting Flask server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


34.125.47.233
✅ MOSQUITO PREDICTOR - READY!

🔗 PUBLIC URL: https://icy-ghosts-exist.loca.lt

📝 Update your HTML files:
   const API_URL = 'https://icy-ghosts-exist.loca.lt';

🔑 Tunnel Password (if asked): Run !curl https://loca.lt/mytunnelpassword

🎯 Test Credentials:
   User:  user@mosquito.com / user123
   Admin: admin@mosquito.com / admin123

🧪 Test Your API:
   https://icy-ghosts-exist.loca.lt/api/health

⚠️  KEEP THIS CELL RUNNING!


INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:27:04] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:29:51] "OPTIONS /api/login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:30:16] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:30:21] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:30:21] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:30:53] "OPTIONS /api/login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:30:53] "POST /api/login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:30:54] "GET /api/dashboard/stats?user_id=2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:30:54] "GET /api/reports?user_id=2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:32:08] "GET /api/dashboard/stats?user_id=2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 11:32:09] "GET /api/reports?user_id=2 HTTP/1.1" 200 -
INFO:werkzeug:127

KeyboardInterrupt: 